<a href="https://colab.research.google.com/github/Sayed-Husain/Image-Classification-Projects/blob/main/Fresh%20Rotten%20Fruits/fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fresh and Rotten Fruits Image Classification with Convolutional Neural Networks from Scratch using TensorFlow and Keras


The objective of the project is to develop a robust image classification system capable of distinguishing between fresh and rotten fruits.






Download dataset or use kaggle API key for access the dataset #dataset link - https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification

In [ ]:

!unzip /content/fruits-fresh-and-rotten-for-classification.zip -d /content

## Load ImageNet Base Model

In [5]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

58889256/58889256 [==============================] - 1s 0us/step


In [6]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [7]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14717766 (56.14 MB)
Trainable params: 3078 (12.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


## Compile Model

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])

## Augment the Data

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)

## Load Dataset

In [11]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('/content/dataset/train',
                                       target_size=(224, 224),
                                       color_mode='rgb',
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('/content/dataset/test',
                                      target_size=(224, 224),
                                      color_mode='rgb',
                                      class_mode="categorical")

Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


## Train the Model

In [12]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=30)

Epoch 1/30
340/340 [==============================] - 273s 759ms/step - loss: 0.9758 - accuracy: 0.7642 - val_loss: 0.2750 - val_accuracy: 0.9018
Epoch 2/30
340/340 [==============================] - 255s 748ms/step - loss: 0.2100 - accuracy: 0.9313 - val_loss: 0.1622 - val_accuracy: 0.9396
Epoch 3/30
340/340 [==============================] - 218s 639ms/step - loss: 0.1348 - accuracy: 0.9542 - val_loss: 0.1284 - val_accuracy: 0.9592
Epoch 4/30
340/340 [==============================] - 216s 634ms/step - loss: 0.0975 - accuracy: 0.9656 - val_loss: 0.0939 - val_accuracy: 0.9711
Epoch 5/30
340/340 [==============================] - 217s 637ms/step - loss: 0.0893 - accuracy: 0.9662 - val_loss: 0.0941 - val_accuracy: 0.9674
Epoch 6/30
340/340 [==============================] - 217s 636ms/step - loss: 0.0775 - accuracy: 0.9723 - val_loss: 0.0830 - val_accuracy: 0.9729
Epoch 7/30
340/340 [==============================] - 216s 634ms/step - loss: 0.0690 - accuracy: 0.9758 - val_loss: 0.0797 -

## Unfreeze Model for Fine Tuning

In [13]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(loss = "categorical_crossentropy",
              optimizer = keras.optimizers.RMSprop(learning_rate = 0.00005),
              metrics = ['accuracy'])

In [14]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=45,
          initial_epoch=30)

Epoch 31/45
340/340 [==============================] - 285s 792ms/step - loss: 0.3299 - accuracy: 0.9275 - val_loss: 0.0241 - val_accuracy: 0.9907
Epoch 32/45
340/340 [==============================] - 259s 759ms/step - loss: 0.1151 - accuracy: 0.9712 - val_loss: 0.0113 - val_accuracy: 0.9956
Epoch 33/45
340/340 [==============================] - 254s 745ms/step - loss: 0.1007 - accuracy: 0.9807 - val_loss: 0.0058 - val_accuracy: 0.9981
Epoch 34/45
340/340 [==============================] - 257s 754ms/step - loss: 0.1101 - accuracy: 0.9805 - val_loss: 0.0111 - val_accuracy: 0.9959
Epoch 35/45
340/340 [==============================] - 258s 758ms/step - loss: 0.0720 - accuracy: 0.9850 - val_loss: 0.0152 - val_accuracy: 0.9963
Epoch 36/45
340/340 [==============================] - 256s 752ms/step - loss: 0.1070 - accuracy: 0.9847 - val_loss: 0.3401 - val_accuracy: 0.9581
Epoch 37/45
340/340 [==============================] - 255s 747ms/step - loss: 0.0905 - accuracy: 0.9835 - val_loss: 0

## Evaluate the Model

In [15]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

84/84 [==============================] - 41s 488ms/step - loss: 0.0130 - accuracy: 0.9963


[0.013014128431677818, 0.9962935447692871]

#Conclusion                                                                   
  In conclusion, the Fresh and Rotten Fruits Image Classification Convolutional Neural Network, developed from scratch using TensorFlow and Keras, achieves outstanding performance with a remarkable 99.63% accuracy and minimal categorical cross-entropy loss of 0.0130. These results underscore the model's proficiency in learning intricate patterns, confirming its effectiveness in accurately classifying fresh and rotten fruits across diverse categories.